In [1]:
from fcc import *

---
Read in entire cerebellum dataset to `pita` object

In [ ]:
%time cerebellum_slide = pita.from_file(datafile='MappedDGE_filteredgenes.csv.gz', beadfile='BeadLocationsForR.csv.gz', data_type='counts')

Subset cerebellum `pita` by pixel location for manageable image size

In [ ]:
%time cerebellum_slide.snippet(xmin=1500, xmax=3000, ymin=1000, ymax=3000)

In [ ]:
cerebellum_slide.data['counts'].shape

In [ ]:
%time cerebellum_slide.map_pixels()

In [ ]:
%time aldoc = cerebellum_slide.assemble_pita(data_type='counts', feature='Aldoc', plot_out=True)

In [ ]:
%%time 
cerebellum_slide.fcc_PCA(data_type='counts', n_components=10, transform='arcsinh')
cerebellum_slide.plot(data_type='PCA')

In [ ]:
%%time 
cerebellum_slide.fcc_UMAP(data_type='PCA', perplexity=30, seed=18)
cerebellum_slide.plot(data_type='UMAP', dim_name='UMAP')

In [ ]:
PC1 = cerebellum_slide.assemble_pita(data_type='PCA', feature=0, plot_out=True)

In [ ]:
cerebellum_slide.plot(data_type='slide-seq', color=cerebellum_slide.data['PCA'][:,0])

In [ ]:
cerebellum_slide.data['slide-seq'].shape

In [ ]:
cerebellum_slide.pixel_map.shape

In [ ]:
# determine pixel bounds from bead locations
xmin = np.floor(cerebellum_slide.data['slide-seq']['xcoord'].min())
xmax = np.ceil(cerebellum_slide.data['slide-seq']['xcoord'].max())
ymin = np.floor(cerebellum_slide.data['slide-seq']['ycoord'].min())
ymax = np.ceil(cerebellum_slide.data['slide-seq']['ycoord'].max())

# define grid for pixel space
grid_x, grid_y = np.mgrid[xmin:xmax, ymin:ymax]

In [ ]:
grid_x